## 7. 동적 계획법

- 분할 정복 기법과 매우 유사하면서, <b>같은 문제를 다시 풀지 않도록 하는 것이 핵심</b> 
- 부분 문제의 답을 저장하기 위해 추가메모리를 사용하므로, 공간으로 시간벌기 + 분할정복 
- 기반상황 설정 후 일반상황 고려

- 겹치는 부분 문제(overlapping subproblem), 최적 부분 구조(optimal substructure, 부분 문제의 최적해를 이용해 전체문제의 최적해를 구할 수 있는 구조)

### 7.1 피보나치 수열과 동적계획법
- 메모이제이션(memoization)
    - 한번 계산한 값을 저장해두었다가 사용하는 최적화 기법
    - top-down
    - O(N)의 속도, O(N)의 공간 필요
    - subprocess의 순서를 잘 모를 때
- 테이블화
    - 저장하는 것은 같으나, 메모리의 항목을 순서적으로 채워나가는 데 초점
    - bottom-up
    - O(N)의 속도, O(N)의 공간 필요
    - 순환호출 부담이 없고 전역메모리 필요없으므로 모듈화에도 유리

In [3]:
#memoization 
n = 8
mem = [None]*(n+1)
def fib_dp_mem(n):
    if (mem[n] == None):
        if n<2:
            mem[n]=n
        else:
            mem[n] = fib_dp_mem(n-1) + fib_dp_mem(n-2)
    return mem[n]

fib_dp_mem(8), mem

(21, [0, 1, 1, 2, 3, 5, 8, 13, 21])

In [9]:
#tabulation
def fib_dp_tab(n):
    f = [None]*(n+1)
    f[0]=0
    f[1]=1
    for i in range(2,n+1):
        f[i] = f[i-1] +f[i-2]
    return f[n], f
fib_dp_tab(8)

(21, [0, 1, 1, 2, 3, 5, 8, 13, 21])

### 7.2 이항계수 구하기

In [ ]:
#분할 정복
def bino_coef_dc(n,r):
    if r==0 or r == n:
        return 1
    return bino_coef_dc(n-1,r-1)+bino_coef_dc(n-1,r)

In [21]:
#tabulation
def bino_coef_dp(n,r):
    C = [[0]*(n+1) for _ in range(n+1)]

    for i in range(n+1):
        for j in range(min(i,r)+1):
            if j==0 or j == i:
                C[i][j] = 1
            else:
                C[i][j] = C[i-1][j-1] + C[i-1][j]

    [print(z) for z in C]
    return C[n][r]
bino_coef_dp(6,5)

[1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0]
[1, 2, 1, 0, 0, 0, 0]
[1, 3, 3, 1, 0, 0, 0]
[1, 4, 6, 4, 1, 0, 0]
[1, 5, 10, 10, 5, 1, 0]
[1, 6, 15, 20, 15, 6, 0]


6

### 7.3 0-1 Knapsack 배낭 채우기 문제

In [23]:
val = [60, 100, 190, 120, 200, 150]
wt = [2, 5, 8, 4, 7, 6]
W = 18
n = len(val)

#분할 정복
def knapSack_dc(W, wt, val, n):
    if n==0 or W == 0:
        return 0
    
    if wt[n-1]>W:
        return knapSack_dc(W, wt, val, n-1)
    else:
        valwithout = knapSack_dc(W, wt, val, n-1)
        valWith = val[n-1] + knapSack_dc(W-wt[n-1], wt, val, n-1)
        return max(valWith, valwithout)
    
knapSack_dc(W, wt, val, n)

480

In [26]:
#동적계획법
def knapSack_dp(W, wt, val, n):
    A = [[0 for _ in range(W+1)] for _ in range(n+1)]

    for i in range(1, n+1):
        for w in range(1, W+1):
            if wt[i-1] > w:
                A[i][w] = A[i-1][w]
            else:
                valWith = val[i-1] + A[i-1][w-wt[i-1]]
                valWithout = A[i-1][w]
                A[i][w] = max(valWith, valWithout)
    [print(i) for i in A]
    return A[n][W]

knapSack_dp(W, wt, val, n)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]
[0, 0, 60, 60, 60, 100, 100, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160]
[0, 0, 60, 60, 60, 100, 100, 160, 190, 190, 250, 250, 250, 290, 290, 350, 350, 350, 350]
[0, 0, 60, 60, 120, 120, 180, 180, 190, 220, 250, 280, 310, 310, 370, 370, 370, 410, 410]
[0, 0, 60, 60, 120, 120, 180, 200, 200, 260, 260, 320, 320, 380, 380, 390, 420, 450, 480]
[0, 0, 60, 60, 120, 120, 180, 200, 210, 260, 270, 320, 330, 380, 380, 410, 420, 470, 480]


480

### 7.4 최장 공통 부분순서 문제(LCS, Longest Common Subsequence)
- 데이터의 유사도 평가에 매우 유용 (유전자 염기 서열 분서이나, 두 소스파일의 차이 찾기)

In [27]:
#순환구조 (정신나감)
def lcs_recur(X, Y, m, n):
    if m==0 or n==0:
        return 0
    elif X[m-1] == Y[n-1]:
        return 1+ lcs_recur(X,Y,m-1,n-1)
    else:
        return max(lcs_recur(X,Y,m,n-1),lcs_recur(X,Y,m-1,n))

In [3]:
#동적계획법

def lcs_dp(X, Y):
    m = len(X)
    n = len(Y)
    L = [[None]*(n+1) for _ in range(m+1)]

    for i in range(m+1):
        for j in range(n+1):
            if i==0 or j==0:
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] =  L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])

    print(lcs_dp_traceback("GAME OVER", "HELLO WORLD", L))
    return L[m][n]


def lcs_dp_traceback(X, Y, L):
    lcs = ""
    i = len(X)
    j = len(Y)
    while i>0 and j>0:
        v = L[i][j]
        if v>L[i][j-1] and v>L[i-1][j] and v>L[i-1][j-1]:
            i-=1
            j-=1
            lcs = X[i] + lcs
        
        elif v == L[i][j-1] and v>L[i-1][j]:
            j -=1
        else:
            i -= 1
    return lcs

lcs_dp("GAME OVER", "HELLO WORLD")

E OR


4

### 7.5 그래프의 인접 행렬 표현과 최단 경로 문제
1. 시작 -> 도착 최단경로
2. 시작 -> 모든 최단경로(Dijkstra)
3. 모두 -> 모두 최단경로(2의 반복)

### 7.6 모든 정점간의 최단 경로 길이
- 3번의 더 간단한 해결법.. Floyd-Warshall 알고리즘
- 

In [4]:
import copy
def shortest_path_floyd(vertex, W):
    vsize = len(vertex)
    D = copy.deepcopy(W)

    for k in range(vsize):
        for i in range(vsize):
            for j in range(vsize):
                if (D[i][k]+D[k][j] < D[i][j]):
                    D[i][j] = D[i][k] + D[k][j]
        printD(D)

def printD(D):
    vsize = len(D)
    print("="*30)
    for i in range(vsize):
        for j in range(vsize):
            if (D[i][j] == INF):
                print("INF ", end='')
            else:
                print("%4d"%D[i][j], end='')
        print("")

INF = 99999
vertex = ['A','B','C','D','E','F','G']
weight = [[0, 7, INF, INF, 3, 10, INF],
          [7, 0, 4, 10, 2, 6, INF],
          [INF, 4, 0, 2, INF, INF,INF],
          [INF, 10, 2, 0 ,11, 9, 4],
          [3, 2, INF, 11, 0, 13, 5],
          [10, 6, INF, 9, 13, 0, INF],
          [INF,INF,INF,4, 5, INF, 0]]
shortest_path_floyd(vertex,weight)


   0   7INF INF    3  10INF 
   7   0   4  10   2   6INF 
INF    4   0   2INF INF INF 
INF   10   2   0  11   9   4
   3   2INF   11   0  13   5
  10   6INF    9  13   0INF 
INF INF INF    4   5INF    0
   0   7  11  17   3  10INF 
   7   0   4  10   2   6INF 
  11   4   0   2   6  10INF 
  17  10   2   0  11   9   4
   3   2   6  11   0   8   5
  10   6  10   9   8   0INF 
INF INF INF    4   5INF    0
   0   7  11  13   3  10INF 
   7   0   4   6   2   6INF 
  11   4   0   2   6  10INF 
  13   6   2   0   8   9   4
   3   2   6   8   0   8   5
  10   6  10   9   8   0INF 
INF INF INF    4   5INF    0
   0   7  11  13   3  10  17
   7   0   4   6   2   6  10
  11   4   0   2   6  10   6
  13   6   2   0   8   9   4
   3   2   6   8   0   8   5
  10   6  10   9   8   0  13
  17  10   6   4   5  13   0
   0   5   9  11   3  10   8
   5   0   4   6   2   6   7
   9   4   0   2   6  10   6
  11   6   2   0   8   9   4
   3   2   6   8   0   8   5
  10   6  10   9   8   0  13
   8   7   6  

### 7.7 편집 거리
- 하나의 문자열을 수정하여 다른 문자열을 만드는 문제
- 삽입, 삭제, 대체 3가지만을 이용함
- 단어 철자오류, 유전자 유사도 측정 등..

In [ ]:
#분할 정복
def edit_distance(S, T, m, n):
    if m==0: return n
    if n==0: return m

    if S[m-1] == T[n-1]:
        return edit_distance(S, T, m-1, n-1)
    
    return 1+ min(edit_distance(S,T,m,n-1),
                  edit_distance(S,T,m-1,n),
                  edit_distance(S,T,m-1,n-1))

In [7]:
#테이블 생성이 애매... memoization

def edit_distance_mem(S, T, m, n):
    global mem
    if m==0: return n
    if n==0: return m

    if mem[m-1][n-1] == None:
        if S[m-1] == T[n-1]:
            mem[m-1][n-1] = edit_distance_mem(S,T,m-1,n-1)
        
        else:
            mem[m-1][n-1] = 1+ min(edit_distance_mem(S,T,m,n-1),
                  edit_distance_mem(S,T,m-1,n),
                  edit_distance_mem(S,T,m-1,n-1))
        print("mem[%d][%d] = "%(m-1,n-1), mem[m-1][n-1])
    
    return mem[m-1][n-1]

S = "tuesday"
T = "thursday"
m = len(S)
n = len(T)
mem = [[None for _ in range(n)] for _ in range(m)]

edit_distance_mem(S,T,m,n)

mem[0][0] =  0
mem[1][0] =  1
mem[2][0] =  2
mem[0][1] =  1
mem[1][1] =  1
mem[2][1] =  2
mem[1][2] =  1
mem[2][2] =  2
mem[0][2] =  2
mem[0][3] =  3
mem[1][3] =  2
mem[2][3] =  2
mem[3][4] =  2
mem[4][5] =  2
mem[5][6] =  2
mem[6][7] =  2


2

## Practice

In [7]:
#1
# def fib_mem(n):
#     global mem
#     if n == 1:
#         return 1
#     elif n == 2:
#         return 2

    
#     if mem[n-1] is None:
#         if mem[n-2] is None :
#             mem[n-2] = fib_mem(n-1)
#         if mem[n-3] is None:
#             mem[n-3] = fib_mem(n-2)
#         mem[n-1] = mem[n-2]+mem[n-3]
    
#     return mem[n-1]

# #memoization 
# n = 8
# mem = [None]*(n+1)
# def fib_dp_mem(n):
#     if (mem[n] == None):
#         if n<2:
#             mem[n]=n
#         else:
#             mem[n] = fib_dp_mem(n-1) + fib_dp_mem(n-2)
#     return mem[n]

# fib_dp_mem(8), mem

def fib_tbl(n):
    t = [0]*(n+1)
    t[0]=0
    t[1]=1
    for i in range(2,n):
        t[i] = t[i-1]+t[i-2]
    
    return t[n-1]
for i in range(1,10):
    print(fib_tbl(i))


0
1
1
2
3
5
8
13
21
